In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
os.chdir('..')
import proppy as pp

Propagator initialized
Propagator initialized
Propagator initialized


## Simulation setup

The software is modular. After initializing the simulation, we can add the indvidual modules as needed.
Important modules are:
- source
- magnetic field
- propagator
- observer

Each module contains special classes for different use cases. In the first tutorial, we only use basic classes to demonstrate the basic usage of the propagation software. In the following tutorials, each module with its special classes will be explained in detail. For example, there exist many different observer types such as time evolution observers (lin or log spcaing) and sphercial observers. 

In [2]:
sim = pp.Simulation()

start simulation


### Source 

First, we have to define a source of particles that we want to propagate. The simples source type is the point source that emmits particles isotropically. The only user-specified parameters are:

- **energy**: Energy of the particles in eV.
- **source posistion**: The position of the point source.
- **number of particles**: Number of particles that should be emitted from this source.

The source can be easily added to the simulation. Afterwards, calling the description on the source of the simulation prints all relevant information and the values of the source parameters.

In [3]:
nr_particles = 10**3
source_pos = np.array([0.0, 0.0, 0.0], dtype=np.float32)
energy = 10**15 # eV
source = pp.PointSourceIsotropicPhi(energy, source_pos, nr_particles)
sim.add_source(source)
sim.source.get_description()

Description Source:
                The source defines the start conditions of the particles 
                and covers the position, direction, energy, etc

position:  [0. 0. 0.]
number particles:  1000
energy:  1000000000000000  eV
source tpye: PointSourceIsotropic


### Propagator

Propagates particles via a correlated rrandom walk. The overall behaviour is governed by a generalized telegraph equation: $\frac{\partial f}{\partial t} = \sum_i \tau_i \frac{\partial^2 f}{\partial t^2} = \sum_i \kappa_i \frac{\partial^2 f}{\partial x_i^2}$.

Here, we use isotropic diffusion for simplicity that propagates particles using a correlated random walk in Cartesian coordinates. Isotropic diffusion is given when the turbulence is isotropic and there is no background field. In the following tutorials also the anisotropic diffusion is used. The only user-specified parameters for the simple case of isotropic diffusion are:

- **mean-free paths**: The mean-free paths $\lambda_i$ can be derived from the diffusion coefficients $\kappa_i$ via $\lambda_i = 3 \kappa_i/c$. Therfore, the diagonal elements of the diagonalized diffusion tensor are needed to determine the characteristics of the transport. For isotropic diffusion all diffusion coefficients are equal $\kappa = [\kappa_{xx}, \kappa_{yy}, \kappa_{zz}]$ with $\kappa_{xx}=\kappa_{yy}=\kappa_{zz}$. Typical free-mean paths of charged particles in plasmoids in AGN jets are $10^{12}$m (see Reichherzer et al. (2021)).
- **number of steps**: The number of simulation steps for each individual particle.
- **step size**: Size of an individual step. Together with the parameter number of steps, the step size determines the trajectory length of the particles. 

The propagator can be easily added to the simulation. Afterwards, calling the description on the propagator of the simulation prints all relevant information and the values of the propagation parameters.

In [ ]:
nr_steps = 10**4
step_size = 0.2*10**10 # [m]
diffusion_coefficient = 5*10**18 # [m^2/s]
speed_of_light = 3*10**8 # [m/s]
mfp_iso = 3*diffusion_coefficient/speed_of_light
mfp = np.array([mfp_iso, mfp_iso, mfp_iso], dtype=np.float32)  # [m]

propagator = pp.IsotropicPropagator(mfp, nr_steps, step_size)
sim.add_propagator(propagator)
sim.propagator.get_description()

### Observer

The Observer determines during the simulation when and what data to write out (observe).

In each simulation step, the current particle state is evaluated by the Observer to check
if one of the observing contions is satisfyed. The conditions to observe can be based 
on the time (-> step) or the coordinates of the particle.

The conditions to observe can be based on the time (or step) or the coordinates of 
    the particle.
    
- step number [unique_steps] -> time (TimeEvolutionObservers)
- radius of observer sphere [shperes] -> sphere around source (SphericalObservers)
- cartesian coordinates [box_dimensions] -> box around source (BoxObserver) (not yet implemented)
   
All special observer will create an Observer object and specify the relevant parameters
for the observation conditions (unique_steps, shperes, box_dimensions)

In [ ]:
substeps = [False, False, True] # observe only steps (no substeps)
min_step = 1
max_step = nr_steps
nr_obs_steps = 600

observer = pp.TimeEvolutionObserverLog(min_step, max_step, nr_obs_steps, substeps)

sim.add_observer(observer)
sim.observer.get_description()

In [ ]:
%time sim.run_simulation()
sim.save_data('data/data_tut_1')

## Analyze statistics

#### Diffusion coefficients

As the particles propagate via a random walk, statistical properties 
of many particles are interesting, such as the diffusion coefficients and
particle distributions. These quantities can be compared to analytical 
predictions.

Running diffusion coefficients are computed with the mean-squared displacement method:

$\kappa_{ii}(t) = \frac{<\Delta x_i^2>}{2t}$.

In [ ]:
df = pd.read_pickle("data/data_tut_1.pkl")

sta = pp.Statistics(df)
isotropic = True # diffusion is isotropic
errors = False # don't show error bars
df_kappas = sta.plot_diffusion_coefficients(isotropic, errors, None)
print('input kappa:', f"{float(diffusion_coefficient):.3}", 'm²/s')
n = 10
print('kappa_{xx}:', f"{np.mean(df_kappas['kappa_xx'][-n:]):.3}", 'm²/s', '+-', f"{np.std(df_kappas['kappa_xx'][-n:]):.3}", 'm²/s')
print('kappa_{yy}:', f"{np.mean(df_kappas['kappa_yy'][-n:]):.3}", 'm²/s', '+-', f"{np.std(df_kappas['kappa_yy'][-n:]):.3}", 'm²/s')
print('kappa_{zz}:', f"{np.mean(df_kappas['kappa_zz'][-n:]):.3}", 'm²/s', '+-', f"{np.std(df_kappas['kappa_zz'][-n:]):.3}", 'm²/s')

## Analyze trajectories

Individual trajectories visualize the correlated random walk of individual particles. The next tutorial focusses on visualizing trajectories in anisotropic diffusion.

In [ ]:
tra = pp.Trajectory(df)
particle_ids = tra.get_particle_ids()
nr_steps = -1 #show all steps
tra.plot_trajectory('x', 'y', 'd', particle_ids[0], nr_steps, None)
tra.plot_trajectory('d', 'z', 'd', particle_ids[0], nr_steps, None)